<a href="https://colab.research.google.com/github/zehgobbes/Dive-in-Deep-Learning/blob/main/ExtractSUS_v1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 ExtractSUS
Download automatizado de dados do DATASUS.

Este notebook contém código aberto para download de dados dos Sistemas de Informação em Saúde (SIS) do DATASUS de forma simples e intuitiva.

**Sistemas disponíveis:**
- **SIM** - Sistema de Informações sobre Mortalidade
- **SINASC** - Sistema de Informações sobre Nascidos Vivos
- **SINAN** - Sistema de Informação de Agravos de Notificação
- **SIA** - Sistema de Informações Ambulatoriais
- **SIH** - Sistema de Informações Hospitalares

---

## 📋 Instruções

1. Pressione o botão *Run all* para carregar a interface.
2. Permita acesso ao drive.
3. Role o cursor até o final do código.
4. Selecione os parâmetros desejados.
5. Clique no botão **"Baixar Dados"** para iniciar o download.
6. Os arquivos serão baixados automaticamente para o seu computador.

**Documentação da biblioteca utilizada: PySUS** | [Documentação](https://github.com/AlertaDengue/PySUS)

In [2]:
# Instalando as bibliotecas necessárias
!pip install -q -U git+https://github.com/AlertaDengue/PySUS.git
!pip install -q ipywidgets

print("✅ Bibliotecas instaladas com sucesso!")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/3

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os
import shutil
from pysus import SIM, SINASC, SINAN, SIA, SIH


# Função que retorna o conjunto de anos com base nos sistemas/grupos selecionados
def compute_years():
    anos = set()

    sistemas = list(sistema_widget.value)

    # SIM: depende dos grupos CID9/CID10 selecionados
    if 'SIM' in sistemas:
        # grupo_sim_widget armazena tuplas (label, code) se você tiver usado assim;
        # no seu código options = GRUPOS['SIM'] => valores são tuplas ('Declaração...', 'CID10')
        # Então grupo_sim_widget.value conterá as tuplas selecionadas; pegamos o segundo elemento.
        for g in list(grupo_sim_widget.value):
            # se for tupla (label, code)
            code = g[1] if isinstance(g, tuple) and len(g) >= 2 else g
            if code == 'CID10' or code == 'CID-10' or code == 'CID10':
                anos |= set(range(1996, 2026))
            if code == 'CID9' or code == 'CID-9' or code == 'CID9':
                anos |= set(range(1979, 1996))

    # SINASC
    if 'SINASC' in sistemas:
        anos |= set(range(1994, 2026))

    # SINAN (base conservadora: 1994 em diante — ajuste se desejar)
    if 'SINAN' in sistemas:
        anos |= set(range(1994, 2026))

    # SIA
    if 'SIA' in sistemas:
        anos |= set(range(1994, 2026))

    # SIH (você mencionou 1984)
    if 'SIH' in sistemas:
        anos |= set(range(1984, 2026))

    # Se nada selecionado, use faixa padrão (por exemplo 1996-2025)
    if not anos:
        return list(range(1996, 2026))

    return sorted(anos)


# Função que atualiza as opções do ano_widget ao mudar seleções
def update_years(change=None):
    new_options = compute_years()
    # Preserve seleção atual quando possível:
    current = set(ano_widget.value) if ano_widget.value else set()
    # Atualiza opções — precisa ser uma lista
    ano_widget.options = new_options
    # Reaplica seleção limitada aos valores válidos
    ano_widget.value = tuple(sorted(v for v in current if v in new_options))



# Definindo os dados para os dropdowns
SISTEMAS = ['SIM', 'SINASC', 'SINAN', 'SIA', 'SIH']

ESTADOS = [
    'AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA',
    'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN',
    'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO'
]

# PARA ANOS FIXOS:
# ANOS = list(range(1996, 2026))  # De 1996 até 2025

# MESES = list(range(1, 13))  # De 1 a 12
# MESES = [('jan','1'),
#          ('fev','2'),
#          ('mar','3'),
#          ('abr','4'),
#          ('mai','5'),
#          ('jun','6'),
#          ('jul','7'),
#          ('ago','8'),
#          ('set','9'),
#          ('out','10'),
#          ('nov','11'),
#          ('dez','12')
#          ],
MESES = [('jan','1'),
         ('fev','2'),
         ('mar','3'),
         ('abr','4'),
         ('mai','5'),
         ('jun','6'),
         ('jul','7'),
         ('ago','8'),
         ('set','9'),
         ('out','10'),
         ('nov','11'),
         ('dez','12')
         ]


# Grupos por sistema
GRUPOS = {
    'SIM': [('Declaração de Óbito (DO)(CID10)','CID10'),
            ('Declaração de Óbito de Recém-nascido (DOR)(CID9)','CID9')],

    'SINASC': [('Declarações de Nascidos Vivos (DN)', 'DN'),
               ('Dados dos Nascidos Vivos por UF de residência (DNR)', 'DNR')],

    'SINAN': [('Acidente de trabalho com material biológico (ACBI)','ACBI'),
              ('Acidente de trabalho (ACGR)','ACGR'),
              ('Acidente por Animais Peçonhentos (ANIM)','ANIM'),
              ('Atendimento Antirrabico (ANTR)','ANTR'),
              ('Botulismo (BOTU)','BOTU'),
              ('Cancêr relacionado ao trabalho (CANC)','CANC'),
              ('Doença de Chagas Aguda (CHAG)','CHAG'),
              ('Febre de Chikungunya (CHIK)','CHIK'),
              ('Cólera (COLE)','COLE'),
              ('Coqueluche (COQU)','COQU'),
              ('Dengue (DENG)','DENG'),
              ('Dermatoses ocupacionais (DERM)','DERM'),
              ('Difteria (DIFT)','DIFT'),
              ('Esquistossomose (ESQU)','ESQU'),
              ('Doença exantemáticas (EXAN)','EXAN'),
              ('Febre Maculosa (FMAC)','FMAC'),
              ('Febre Tifóide (FTIF)','FTIF'),
              ('Hanseníase (HANS)','HANS'),
              ('Hantavirose (HANT)','HANT'),
              ('Hepatites Virais (HEPA)','HEPA'),
              ('Intoxicação Exógena (IEXO)','IEXO'),
              ('Influenza Pandêmica (INFL)','INFL'),
              ('Leishmaniose Visceral (LEIV)','LEIV'),
              ('Leptospirose (LEPT)','LEPT'),
              ('LER/Dort (LERD)','LERD'),
              ('Leishmaniose Tegumentar Americana (LTAN)','LTAN'),
              # (' (LTAV)',''),
              ('Malária (MALA)','MALA'),
              ('Meningite (MENI)','MENI'),
              ('Transtornos mentais relacionados ao trabalho (MENT)','MENT'),
              ('Notificação de Tracoma (NTRA)','NTRA'),
              ('Perda auditiva por ruído relacionado ao trabalho (PAIR)','PAIR'),
              ('Peste (PEST)','PEST'),
              ('Paralisia Flácida Aguda (PFAN)','PFAN'),
              ('Pneumoconioses realacionadas ao trabalho (PNEU)','PNEU'),
              ('Raiva (RAIV)','RAIV'),
              ('Surto Doenças Transmitidas por Alimentos (SDTA)','SDTA'),
              ('Sífilis Adquirida (SIFA)','SIFA'),
              ('Sífilis Congênita (SIFC)','SIFC'),
              ('Sífilis em Gestante (SIFG)','SIFG'),
              ('Síndrome da Rubéola Congênia (SRC)','SRC'),
              ('Tétano Acidental (TETA)','TETA'),
              ('Tétano Neonatal (TETN)','TETN'),
              ('Toxoplasmose Congênita (TOXC)','TOXC'),
              ('Toxoplasmose Gestacional (TOXG)','TOXG'),
              ('Inquérito de Tracoma (TRAC)','TRAC'),
              ('Tuberculose (TUBE)','TUBE'),
              ('Varicela (VARC)','VARC'),
              ('Violência doméstica, sexual e/ou outras (VIOL)','VIOL'),
              ('ZIKA (ZIKA)','ZIKA')],


    'SIA': [('APAC de Cirurgia Bariátrica (AB)','AB'),
            ('APAC de Acompanhamento Pós Cirurgia Bariátrica (ABO)','ABO'),
            ('APAC de Confecção de Fístula (ACF)','ACF'),
            ('APAC de Laudos Diversos (AD)','AD'),
            ('APAC de Medicamentos (AM)','AM'),
            ('APAC de Acompanhamento Multiprofissional (AMP)','AMP'), # INSERIDO DEPOIS
            ('APAC de Nefrologia (AN)','AN'),
            ('APAC de Quimioterapia (AQ)','AQ'),
            ('APAC de Radioterapia (AR)','AR'),
            ('APAC de Tratamento Dialítico (ATD)','ATD'),
            ('Boletim de Produção Ambulatorial Individualizado (BI)','BI'),
            # ('IMPBO','IMPBO'), # Sem arquivos
            ('Produção Ambulatorial (PA)','PA'),
            # ('PAM','PAM'),  # Sem arquivos
            # ('PAR','PAR'),  # Sem arquivos
            # ('PAS','PAS'), # Sem arquivos
            ('RAAS Psicossocial (PS)','PS'),
            ('RAAS de Atenção Domiciliar (SAD)','SAD')],

    'SIH':  [('AIH Reduzida (RD)', 'RD'),
            ('AIH Rejeitada (RJ)', 'RJ'),
            ('AIH Rejeitada com erro (ER)', 'ER'),
            ('Serviços Profissionais (SP)', 'SP'),
            # ('CM', 'CM'),  # Sem arquivos
            ('Cadastro Hospitalar (CH)', 'CH')]
}



# Estilo CSS
display(HTML("""
<style>
.widget-label {
  font-weight: bold;
  font-size: 14px;
  align-self: flex-start !important;  /* move o texto para o topo esquerdo */
  margin-top: 0px !important;
  padding-top: 0px !important;}
.widget-dropdown select { font-size: 13px; }
.output-box {
    border: 2px solid #4CAF50;
    padding: 15px;
    border-radius: 8px;
    margin-top: 20px;
    background-color: #f9f9f9;
}
</style>
"""))

# Widgets
sistema_widget = widgets.SelectMultiple(
    options=SISTEMAS,
    description='Sistema(s):',
    disabled=False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='400px', height='90px')
)

estado_widget = widgets.SelectMultiple(
    options=ESTADOS,
    description='Estado(s):',
    disabled=False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='400px', height='120px')
)

# Crie ano_widget com opções vazias ou padrão e depois atualize
ano_widget = widgets.SelectMultiple(
    options=compute_years(),  # inicializa com o que é apropriado no momento
    description='Ano(s):',
    disabled=False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='400px', height='120px')
)

mes_widget = widgets.SelectMultiple(
    options=MESES,
    description='Mês(es):',
    disabled=False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='400px', height='120px')
)

# Widgets de grupos (inicialmente ocultos)
grupo_sim_widget = widgets.SelectMultiple(
    options=GRUPOS['SIM'],
    description='Grupo SIM: \n',
    disabled=False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='400px', height='100px', display='none', margin='30px')
)

grupo_sinasc_widget = widgets.SelectMultiple(
    options=GRUPOS['SINASC'],
    description='Grupo SINASC:',
    disabled=False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='400px', height='100px', display='none', margin='30px')
)

grupo_sinan_widget = widgets.SelectMultiple(
    options=GRUPOS['SINAN'],
    description='Grupo SINAN:',
    disabled=False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='400px', height='100px', display='none', margin='30px')   #####
)

grupo_sia_widget = widgets.SelectMultiple(
    options=GRUPOS['SIA'],
    description='Grupo SIA:',
    disabled=False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='400px', height='100px', display='none', margin='30px')   #####
)

grupo_sih_widget = widgets.SelectMultiple(
    options=GRUPOS['SIH'],
    description='Grupo SIH:',
    disabled=False,
    style={'description_width': '150px'},
    layout=widgets.Layout(width='400px', height='100px', display='none', margin='30px')   #####
)

# Função para mostrar/ocultar grupos
def atualizar_grupos(change):
    sistemas_selecionados = list(sistema_widget.value)

    grupo_sim_widget.layout.display = 'block' if 'SIM' in sistemas_selecionados else 'none'
    grupo_sinasc_widget.layout.display = 'block' if 'SINASC' in sistemas_selecionados else 'none'
    grupo_sinan_widget.layout.display = 'block' if 'SINAN' in sistemas_selecionados else 'none'
    grupo_sia_widget.layout.display = 'block' if 'SIA' in sistemas_selecionados else 'none'
    grupo_sih_widget.layout.display = 'block' if 'SIH' in sistemas_selecionados else 'none'

sistema_widget.observe(atualizar_grupos, names='value')

# Observadores: atualiza ao mudar sistema ou grupos
sistema_widget.observe(update_years, names='value')
grupo_sim_widget.observe(update_years, names='value')
grupo_sinasc_widget.observe(update_years, names='value')
grupo_sinan_widget.observe(update_years, names='value')
grupo_sia_widget.observe(update_years, names='value')
grupo_sih_widget.observe(update_years, names='value')

# Chamada inicial para garantir sincronia ao exibirmos os widgets
update_years()


# Botão de download
botao_download = widgets.Button(
    description='📥 Baixar Dados',
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px'),
    style={'font_weight': 'bold'}
)

# Área de saída
output = widgets.Output()

# Função de download
def baixar_dados(b):
    with output:
        clear_output()

        # Validações
        if not sistema_widget.value:
            print("❌ Por favor, selecione pelo menos um sistema!")
            return

        if not estado_widget.value:
            print("❌ Por favor, selecione pelo menos um estado!")
            return

        if not ano_widget.value:
            print("❌ Por favor, selecione pelo menos um ano!")
            return

        # Verificar se grupos foram selecionados
        sistemas_selecionados = list(sistema_widget.value)
        grupos_validos = True

        if 'SIM' in sistemas_selecionados and not grupo_sim_widget.value:
            print("❌ Por favor, selecione pelo menos um grupo para SIM!")
            grupos_validos = False

        if 'SINASC' in sistemas_selecionados and not grupo_sinasc_widget.value:
            print("❌ Por favor, selecione pelo menos um grupo para SINASC!")
            grupos_validos = False

        if 'SINAN' in sistemas_selecionados and not grupo_sinan_widget.value:
            print("❌ Por favor, selecione pelo menos um grupo para SINAN!")
            grupos_validos = False

        if 'SIA' in sistemas_selecionados and not grupo_sia_widget.value:
            print("❌ Por favor, selecione pelo menos um grupo para SIA!")
            grupos_validos = False

        if 'SIH' in sistemas_selecionados and not grupo_sih_widget.value:
            print("❌ Por favor, selecione pelo menos um grupo para SIH!")
            grupos_validos = False

        if not grupos_validos:
            return

        print("🔄 Iniciando download dos dados...\n")

        # Preparar variáveis
        my_systems_str = list(sistema_widget.value)
        my_state = list(estado_widget.value)
        my_year = list(ano_widget.value)
        my_month = list(mes_widget.value) if mes_widget.value else list(range(1, 13))

        my_group = {
            'SIM': list(grupo_sim_widget.value),
            'SINASC': list(grupo_sinasc_widget.value),
            'SINAN': list(grupo_sinan_widget.value),
            'SIA': list(grupo_sia_widget.value),
            'SIH': list(grupo_sih_widget.value)
        }

        meses_selecionados = [MESES[int(mes_widget.value[i])-1][0] for i in range(len(mes_widget.value))]


        print(f"📋 Sistema(s): {', '.join(my_systems_str)}")
        print(f"📍 Estado(s): {', '.join(my_state)}")
        print(f"📅 Ano(s): {', '.join(map(str, my_year))}")
        print("📆 Mes(es):", ', '.join(meses_selecionados))

        # Limpar arquivos anteriores
        caminho_pasta = '/root/pysus'
        if os.path.exists(caminho_pasta):
            shutil.rmtree(caminho_pasta)
            print(f"🗑️ Pasta {caminho_pasta} foi limpa.\n")

        # Importar bibliotecas
        from pysus import SIM, SINASC, SINAN, SIA, SIH
        import pandas as pd
        from google.colab import files
        import pprint

        # -------------
        my_sistemas = [globals()[nome] for nome in my_systems_str if nome in globals()]   # Converte a lista de strings em uma lista de variáveis

        s = 0
        for my_sistema in my_sistemas:
            print(f"\n{'='*60}")
            print(f"🔍 Processando sistema: {my_systems_str[s]}")
            print(f"{'='*60}\n")

            my_sistema = my_sistema().load()

            # Obter arquivos
            if my_sistema.name == 'SIM':
                arquivos = my_sistema.get_files(my_group['SIM'], uf=my_state, year=my_year)
            elif my_sistema.name == 'SINASC':
                arquivos = my_sistema.get_files(my_group['SINASC'], uf=my_state, year=my_year)
            elif my_sistema.name == 'SINAN':
                arquivos = my_sistema.get_files(dis_code=my_group['SINAN'], year=my_year)
            elif my_sistema.name == 'SIA':
                arquivos = my_sistema.get_files(my_group['SIA'], uf=my_state, year=my_year, month=my_month)
            elif my_sistema.name == 'SIH':
                arquivos = my_sistema.get_files(my_group['SIH'], uf=my_state, year=my_year, month=my_month)

            print(f"📦 Total de arquivos encontrados: {len(arquivos)}.\n")

            for d in range(len(arquivos)):
                print(f"⬇️ Baixando arquivo {d+1}/{len(arquivos)}...")
                my_sistema.describe(arquivos[d])
                parquet_list = my_sistema.download(arquivos[d])

                if type(parquet_list) != list:
                    parquet_list = [parquet_list]

                for j in range(len(parquet_list)):
                  try:
                      df = parquet_list[j].to_dataframe()
                      print(f"\n Arquivo {j+1}: carregado com sucesso com {len(df)} linhas. \n")            ############
                  except ValueError:
                      print(f"\n Arquivo {j+1}: .parquet VAZIO ou CORROMPIDO!\n Tente outro grupo! \n")

                lista_arquivos = os.listdir(f'{parquet_list[j]}')

                # Converter para CSV e fazer download
                for i in range(len(lista_arquivos)):
                    df = pd.read_parquet(f'{parquet_list[j]}/{lista_arquivos[i]}')
                    df.to_csv(f'{parquet_list[j]}/{my_systems_str[s]}_{d}_{j}_{i}.csv', index=True)

                print(f"💾 Fazendo download de {len(lista_arquivos)} arquivo(s)...")

                for i in range(len(lista_arquivos)):
                    files.download(f'{parquet_list[j]}/{my_systems_str[s]}_{d}_{j}_{i}.csv')

                for i in range(len(lista_arquivos)):
                    files.download(f'{parquet_list[j]}/{lista_arquivos[i]}')

                print(f"✅ Arquivo {d+1} processado com sucesso!\n")

            s = s + 1

        print(f"\n{'='*60}")
        print("🎉 Download concluído com sucesso!")
        print(f"{'='*60}")

botao_download.on_click(baixar_dados)

# Layout
print("\n")
display(HTML("<h2 style='color: #4CAF50;'>⚙️ Configurações de Download</h2>"))
display(HTML("<p style='font-size: 13px;'><b>Dica:</b> Use Ctrl (ou Cmd no Mac) para selecionar múltiplos itens</p>"))

display(sistema_widget)
display(grupo_sim_widget)
display(grupo_sinasc_widget)
display(grupo_sinan_widget)
display(grupo_sia_widget)
display(grupo_sih_widget)
display(estado_widget)
display(ano_widget)
display(mes_widget)
display(HTML("<br>"))
display(botao_download)
display(output)

SelectMultiple(description='Sistema(s):', layout=Layout(height='90px', width='400px'), options=('SIM', 'SINASC…

SelectMultiple(description='Grupo SIM: \n', layout=Layout(display='none', height='100px', margin='30px', width…

SelectMultiple(description='Grupo SINASC:', layout=Layout(display='none', height='100px', margin='30px', width…

SelectMultiple(description='Grupo SINAN:', layout=Layout(display='none', height='100px', margin='30px', width=…

SelectMultiple(description='Grupo SIA:', layout=Layout(display='none', height='100px', margin='30px', width='4…

...

SelectMultiple(description='Estado(s):', layout=Layout(height='120px', width='400px'), options=('AC', 'AL', 'A…

SelectMultiple(description='Ano(s):', layout=Layout(height='120px', width='400px'), options=(1996, 1997, 1998,…

SelectMultiple(description='Mês(es):', layout=Layout(height='120px', width='400px'), options=(('jan', '1'), ('…

Button(button_style='success', description='📥 Baixar Dados', layout=Layout(height='50px', width='200px'), styl…

Output()

Os arquivos também podem ser acessados em '/root/pysus/', pasta raiz do Google Colab.

Se foram extraídos dados utilizando o ExtractSUS, solicitamos que realize a citação conforme abaixo, incentivando assim a atualização e manutenção do código. Desde já agradecemos!


**bibitex:**

**ABNT:**

**MLA:**

**APA:**

**Chicago:**

**Harvard:**

**Vancouver:**



                                                             Copyright (C) 2025 José Gobbes